# Smart Tracking System Blockchain Ledger - Milestone 1

#### Group:
#### Member:
**Joem Bolinas | Jenny Rose Villaroza | Jennifer Cerio | John Louie Alova | Charivel Palisoc**

##### Section: S3101
---


In this milestone, we will:
1. Connect our Python environment to the deployed LogisticsDataStorage smart contract
2. Load IoT data from our Week 2 simulation (CSV file)
3. Store multiple IoT records on the blockchain
4. Retrieve and verify the stored data
5. Document our complete process for submission

In [1]:
# Install required libraries if not already installed
!pip install web3 pandas

## Milestone 1 Checklist

### Environment Setup
- [ ] Connect to Ganache successfully
- [ ] Load smart contract with correct ABI
- [ ] Verify contract owner permissions

### Data Processing
- [ ] Load IoT CSV data from Week 2
- [ ] Preview and validate data structure
- [ ] Store multiple records on blockchain

### Verification
- [ ] Retrieve stored records
- [ ] Verify transaction hashes
- [ ] Document total records count

---

## Python Script to Interact with the Smart Contract

Below is the script that:
- Connects to Ganache
- Loads the deployed contract using its address and ABI
- Checks the initial number of records
- Stores a dummy IoT data record
- Retrieves and displays the stored record

## Step 1: Load IoT Data from Week 2 Simulation

First, let's load and examine our simulated IoT data from the CSV file created in Week 2.

In [ ]:
import pandas as pd
import os

# Load the CSV data from Week 2
csv_path = "WEEK-2/simulated_logistic_iot_data.csv"

if os.path.exists(csv_path):
    iot_data = pd.read_csv(csv_path)
    print(f"CSV file loaded successfully!")
    print(f"Total records in CSV: {len(iot_data)}")
    print(f"Data columns: {list(iot_data.columns)}")
    
    print("\nFirst 3 records from CSV:")
    print(iot_data.head(3).to_string(index=False))
    
    print("\nData summary:")
    print(f"  - Date range: {iot_data['Timestamp'].min()} to {iot_data['Timestamp'].max()}")
    print(f"  - Unique packages: {iot_data['PackageID'].nunique()}")
    print(f"  - Unique devices: {iot_data['DeviceID'].nunique()}")
    print(f"  - Temperature range: {iot_data['Temperature_C'].min()}°C to {iot_data['Temperature_C'].max()}°C")
else:
    print(f"CSV file not found at: {csv_path}")
    print("Please ensure the Week 2 CSV file exists in the WEEK-2 folder.")
    print("Let's check current directory:")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Files in current directory: {os.listdir('.')}")
    if os.path.exists('WEEK-2'):
        print(f"Files in WEEK-2 directory: {os.listdir('WEEK-2')}")
    else:
        print("WEEK-2 directory not found")

✅ CSV file loaded successfully!
📊 Total records in CSV: 100
📋 Data columns: ['Timestamp', 'PackageID', 'RFIDTag', 'Latitude', 'Longitude', 'Temperature_C', 'DeviceID']

🔍 First 3 records from CSV:
          Timestamp PackageID         RFIDTag  Latitude  Longitude  Temperature_C        DeviceID
2025-05-09 04:49:40   PKG1001 RFID_82074261_A 14.598842 120.984874           3.58 Tracker_Dev_101
2025-05-09 04:51:40   PKG1003 RFID_56419767_A 14.597552 120.988724           5.45 Tracker_Dev_102
2025-05-09 04:56:40   PKG1004 RFID_28192170_A 14.599729 120.989940           4.22 Tracker_Dev_103

📈 Data summary:
  - Date range: 2025-05-09 04:49:40 to 2025-05-09 13:03:40
  - Unique packages: 64
  - Unique devices: 5
  - Temperature range: 3.5°C to 10.27°C


## Step 2: Connect to Ganache and Load Smart Contract

Now we'll establish our blockchain connection and load the smart contract.

In [ ]:
from web3 import Web3
import json
from datetime import datetime

# --- Configuration ---
GANACHE_URL = "http://127.0.0.1:7545"
CONTRACT_ADDRESS = "0xa56e14ab07f139b9fa1569bb1374c0dd088fb039"

print("Starting Milestone 1: Smart Tracking System Blockchain Ledger")
print("=" * 70)

# ABI copied from Remix after compiling the contract
CONTRACT_ABI_JSON = """
[
	{
		"inputs": [],
		"stateMutability": "nonpayable",
		"type": "constructor"
	},
	{
		"anonymous": false,
		"inputs": [
			{
				"indexed": false,
				"internalType": "uint256",
				"name": "blockchainTimestamp",
				"type": "uint256"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "originalTimestamp",
				"type": "string"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "packageId",
				"type": "string"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "rfidTag",
				"type": "string"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "latitude",
				"type": "string"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "longitude",
				"type": "string"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "temperatureC",
				"type": "string"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "deviceId",
				"type": "string"
			}
		],
		"name": "LogisticsDataStored",
		"type": "event"
	},
	{
		"inputs": [
			{
				"internalType": "string",
				"name": "_originalTimestamp",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_packageId",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_rfidTag",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_latitude",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_longitude",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_temperatureC",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_deviceId",
				"type": "string"
			}
		],
		"name": "storeLogisticsData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "getLogisticsRecord",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "blockchainTimestamp",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "originalTimestamp",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "packageId",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "rfidTag",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "latitude",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "longitude",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "temperatureC",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "deviceId",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "getTotalRecords",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "logisticsRecords",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "blockchainTimestamp",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "originalTimestamp",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "packageId",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "rfidTag",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "latitude",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "longitude",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "temperatureC",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "deviceId",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "MAX_ENTRIES",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "owner",
		"outputs": [
			{
				"internalType": "address",
				"name": "",
				"type": "address"
			}
		],
		"stateMutability": "view",
		"type": "function"
	}
]
"""

print("Configuration:")
print(f"  Ganache URL: {GANACHE_URL}")
print(f"  Contract Address: {CONTRACT_ADDRESS}")
print(f"  Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()

# 1. Connect to Ganache
print("Step 1: Connecting to Ganache...")
try:
    web3 = Web3(Web3.HTTPProvider(GANACHE_URL))
    if web3.is_connected():
        print(f"Successfully connected to Ganache at {GANACHE_URL}")
        print(f"Block number: {web3.eth.block_number}")
        print(f"Available accounts: {len(web3.eth.accounts)}")
    else:
        print(f"Connection failed. Please ensure Ganache is running.")
        raise Exception("Ganache connection failed")
except Exception as e:
    print(f"Connection error: {e}")
    print("Make sure Ganache is running on port 7545")
    exit()

# 2. Load Smart Contract
print("\nStep 2: Loading Smart Contract...")
try:
    contract_abi = json.loads(CONTRACT_ABI_JSON)
    contract = web3.eth.contract(address=web3.to_checksum_address(CONTRACT_ADDRESS), abi=contract_abi)
    print(f"Smart contract loaded successfully")
    print(f"Contract address: {CONTRACT_ADDRESS}")
    
    # Verify contract owner
    contract_owner = contract.functions.owner().call()
    print(f"Contract owner: {contract_owner}")
except Exception as e:
    print(f"Contract loading error: {e}")
    exit()

# Set the default sender account (should be the contract owner)
default_sender_account = web3.eth.accounts[0]
web3.eth.default_account = default_sender_account
print(f"Using sender account: {default_sender_account}")

# Verify permissions
if default_sender_account.lower() == contract_owner.lower():
    print("Account has owner permissions")
else:
    print("Warning: Account may not have owner permissions")

# 3. Check initial state
print("\nStep 3: Checking Initial Contract State...")
initial_total_records = contract.functions.getTotalRecords().call()
max_entries = contract.functions.MAX_ENTRIES().call()
print(f"Current records in contract: {initial_total_records}")
print(f"Maximum allowed entries: {max_entries}")
print(f"Available slots: {max_entries - initial_total_records}")

print("\n" + "="*50)
print("ENVIRONMENT SETUP COMPLETE")
print("="*50)

# 4. Store a dummy IoT data entry
dummy_data = {
    "_originalTimestamp": "2025-01-01 12:00:00",
    "_packageId": "PKG_PYTHON_001",
    "_rfidTag": "RFID_PY_TEST",
    "_latitude": "10.12345",
    "_longitude": "120.54321",
    "_temperatureC": "22.5",
    "_deviceId": "Tracker_Py_01"
}

tx_hash = contract.functions.storeLogisticsData(
    dummy_data["_originalTimestamp"],
    dummy_data["_packageId"],
    dummy_data["_rfidTag"],
    dummy_data["_latitude"],
    dummy_data["_longitude"],
    dummy_data["_temperatureC"],
    dummy_data["_deviceId"]
).transact({'from': default_sender_account, 'gas': 1500000})

print(f"Transaction sent. Waiting for receipt... (Hash: {tx_hash.hex()})")
receipt = web3.eth.wait_for_transaction_receipt(tx_hash, timeout=120)
if receipt.status == 1:
    print(f"Dummy data stored successfully! Transaction Hash: {receipt.transactionHash.hex()}")
else:
    print(f"Transaction failed. Status: {receipt.status}")

# 5. Retrieve and display the stored record
current_total_records = contract.functions.getTotalRecords().call()
print(f"Total records after storing: {current_total_records}")

if current_total_records > 0:
    record = contract.functions.getLogisticsRecord(current_total_records - 1).call()
    print("\nRetrieved Record:")
    print(f"  Blockchain Timestamp: {record[0]}")
    print(f"  Original Timestamp:   {record[1]}")
    print(f"  Package ID:           {record[2]}")
    print(f"  RFID Tag:             {record[3]}")
    print(f"  Latitude:             {record[4]}")
    print(f"  Longitude:            {record[5]}")
    print(f"  Temperature C:        {record[6]}")
    print(f"  Device ID:            {record[7]}")
else:
    print("No records found in the contract.")

🚀 Starting Milestone 1: Smart Tracking System Blockchain Ledger
📋 Configuration:
  Ganache URL: http://127.0.0.1:7545
  Contract Address: 0xa56e14ab07f139b9fa1569bb1374c0dd088fb039
  Timestamp: 2025-05-29 05:51:04

🔗 Step 1: Connecting to Ganache...
✅ Successfully connected to Ganache at http://127.0.0.1:7545
📊 Block number: 104
🏦 Available accounts: 10

📜 Step 2: Loading Smart Contract...
✅ Smart contract loaded successfully
📍 Contract address: 0xa56e14ab07f139b9fa1569bb1374c0dd088fb039
👑 Contract owner: 0xB3B75FA814041f3176d4812324CD47A0C50F31A6
📝 Using sender account: 0xB3B75FA814041f3176d4812324CD47A0C50F31A6
✅ Account has owner permissions

📊 Step 3: Checking Initial Contract State...
📈 Current records in contract: 2
🔢 Maximum allowed entries: 100
💾 Available slots: 98

🎯 ENVIRONMENT SETUP COMPLETE
Transaction sent. Waiting for receipt... (Hash: 25751d309d07f030d2662d81cc7702ed614fd559c233fc9921b26e7c4902abd0)
✅ Dummy data stored successfully! Transaction Hash: 25751d309d07f030d26

## Step 3: Store IoT Data on Blockchain

Now we'll store our CSV data records on the blockchain. We'll start with a few records to demonstrate the process.

In [ ]:
# Store IoT data from CSV on blockchain
print("Step 4: Storing IoT Data on Blockchain")
print("=" * 50)

# Select first 5 records to store (to stay within demo limits)
records_to_store = min(5, len(iot_data))
print(f"Storing first {records_to_store} records from CSV...")
print()

# Track transaction information
transaction_hashes = []
stored_records = []

for i in range(records_to_store):
    row = iot_data.iloc[i]
    
    print(f"Processing record {i+1}/{records_to_store}: {row['PackageID']}")
    
    try:
        # Store data on blockchain
        tx_hash = contract.functions.storeLogisticsData(
            str(row['Timestamp']),
            str(row['PackageID']),
            str(row['RFIDTag']),
            str(row['Latitude']),
            str(row['Longitude']),
            str(row['Temperature_C']),
            str(row['DeviceID'])
        ).transact({'from': default_sender_account, 'gas': 1500000})
        
        print(f"  Transaction sent: {tx_hash.hex()}")
        
        # Wait for transaction receipt
        receipt = web3.eth.wait_for_transaction_receipt(tx_hash, timeout=120)
        
        if receipt.status == 1:
            print(f"  Transaction successful!")
            transaction_hashes.append(tx_hash.hex())
            stored_records.append({
                'index': i,
                'package_id': row['PackageID'],
                'tx_hash': tx_hash.hex(),
                'block_number': receipt.blockNumber,
                'gas_used': receipt.gasUsed
            })
        else:
            print(f"  Transaction failed with status: {receipt.status}")
            
    except Exception as e:
        print(f"  Error storing record: {e}")
    
    print()

print("Storage Summary:")
print(f"  Successfully stored: {len(stored_records)} records")
print(f"  Transaction hashes: {len(transaction_hashes)}")

if transaction_hashes:
    print(f"  First transaction hash: {transaction_hashes[0]}")
    print(f"  Last transaction hash: {transaction_hashes[-1]}")

print("\n" + "="*50)
print("DATA STORAGE COMPLETE")
print("="*50)

🚀 Step 4: Storing IoT Data on Blockchain
📝 Storing first 5 records from CSV...

📦 Processing record 1/5: PKG1001
  ⏳ Transaction sent: cc84e1689da6a5dc670cd556a063568f8a79e4f7f717c2ad7c7a20baf3f8c546
  ✅ Transaction successful!

📦 Processing record 2/5: PKG1003
  ⏳ Transaction sent: 8ff35a9e548825cf7fcf004a0cfee8529639e4e61f9d7598d4efd8c51c0955f9
  ✅ Transaction successful!

📦 Processing record 3/5: PKG1004
  ⏳ Transaction sent: 3678c335fb1f934de5a6fe784b3831801ea092e23e718289525c2d3fa2f1c264
  ✅ Transaction successful!

📦 Processing record 4/5: PKG1007
  ⏳ Transaction sent: e627f68c5095d424979d426780951377a34c550bd3115d326083a5ea787d4c36
  ✅ Transaction successful!

📦 Processing record 5/5: PKG1005
  ⏳ Transaction sent: 709ed0513db8a0494900a4df2efc39206991a65b1a1a4ecc65725759350aaa48
  ✅ Transaction successful!

📊 Storage Summary:
  ✅ Successfully stored: 5 records
  🔗 Transaction hashes: 5
  🥇 First transaction hash: cc84e1689da6a5dc670cd556a063568f8a79e4f7f717c2ad7c7a20baf3f8c546
  

## Step 4: Retrieve and Verify Stored Data

Let's verify that our data was stored correctly by retrieving it from the blockchain.

In [ ]:
# Retrieve and verify stored data
print("Step 5: Retrieving and Verifying Stored Data")
print("=" * 50)

# Get updated total records
current_total_records = contract.functions.getTotalRecords().call()
print(f"Total records now on blockchain: {current_total_records}")
print(f"Records added in this session: {current_total_records - initial_total_records}")
print()

if current_total_records > 0:
    print("Retrieved Records from Blockchain:")
    print("-" * 80)
    
    # Retrieve the last few records (or all if few)
    records_to_show = min(3, current_total_records)
    
    for i in range(max(0, current_total_records - records_to_show), current_total_records):
        try:
            record = contract.functions.getLogisticsRecord(i).call()
            
            print(f"\nRecord #{i + 1}:")
            print(f"  Blockchain Timestamp: {record[0]} ({datetime.fromtimestamp(record[0]).strftime('%Y-%m-%d %H:%M:%S')})")
            print(f"  Original Timestamp: {record[1]}")
            print(f"  Package ID: {record[2]}")
            print(f"  RFID Tag: {record[3]}")
            print(f"  Coordinates: ({record[4]}, {record[5]})")
            print(f"  Temperature: {record[6]}°C")
            print(f"  Device ID: {record[7]}")
            
        except Exception as e:
            print(f"Error retrieving record {i}: {e}")
    
    # Show first stored record details for milestone
    if current_total_records > 0:
        first_record = contract.functions.getLogisticsRecord(0).call()
        print("\n" + "="*50)
        print("FIRST STORED RECORD (for Milestone)")
        print("="*50)
        print(f"Timestamp: {first_record[0]}")
        print(f"Device ID: {first_record[7]}")
        print(f"Data Type: Logistics Tracking")
        print(f"Data Value: Package {first_record[2]} at {first_record[6]}°C")
        
else:
    print("No records found in the contract.")

print("\n" + "="*50)
print("MILESTONE 1 COMPLETE")
print("="*50)

🔍 Step 5: Retrieving and Verifying Stored Data
📊 Total records now on blockchain: 8
📈 Records added in this session: 6

📋 Retrieved Records from Blockchain:
--------------------------------------------------------------------------------

📦 Record #6:
  🕐 Blockchain Timestamp: 1748469074 (2025-05-29 05:51:14)
  📅 Original Timestamp: 2025-05-09 04:56:40
  📦 Package ID: PKG1004
  🏷️ RFID Tag: RFID_28192170_A
  🌍 Coordinates: (14.599729, 120.98994)
  🌡️ Temperature: 4.22°C
  📱 Device ID: Tracker_Dev_103

📦 Record #7:
  🕐 Blockchain Timestamp: 1748469074 (2025-05-29 05:51:14)
  📅 Original Timestamp: 2025-05-09 05:01:40
  📦 Package ID: PKG1007
  🏷️ RFID Tag: RFID_66393603_B
  🌍 Coordinates: (14.600436, 120.990858)
  🌡️ Temperature: 3.92°C
  📱 Device ID: Tracker_Dev_104

📦 Record #8:
  🕐 Blockchain Timestamp: 1748469075 (2025-05-29 05:51:15)
  📅 Original Timestamp: 2025-05-09 05:07:40
  📦 Package ID: PKG1005
  🏷️ RFID Tag: RFID_56042076_C
  🌍 Coordinates: (14.604527, 120.987365)
  🌡️ Tempera

## Milestone 1 Summary

### Completed Tasks:
1. **Environment Setup**: Successfully connected to Ganache blockchain
2. **Smart Contract Integration**: Loaded and verified LogisticsDataStorage contract
3. **Data Loading**: Imported IoT data from Week 2 CSV file (100 records)
4. **Blockchain Storage**: Stored multiple IoT records as immutable transactions
5. **Data Verification**: Retrieved and verified stored blockchain records

### Results:
- **CSV Records**: 100 IoT sensor records loaded
- **Blockchain Records**: Multiple records successfully stored
- **Transaction Verification**: All transactions confirmed with receipt
- **Data Integrity**: Retrieved data matches original CSV format

### Next Steps:
- Prepare presentation for Week 5 synchronous session
- Document complete process in milestone worksheet
- Submit final Milestone 1 deliverable

---

*This completes the Milestone 1 requirements for MO-IT148 Smart Tracking System Blockchain Ledger project.*